# Text Classification
텍스트 문서의 다양한 분류방법에 대해 학습

### 데이터 준비 (복습)
이전에 사용한 영화리뷰 데이터를 이용해서 주어진 리뷰 내용에 대해 positive와 negative를 분류하는 분류기를 학습하고자 함

In [1]:
from nltk.corpus import movie_reviews
fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 
#file id를 이용해 label로 사용할 category 즉 positive와 negative 정보를 순서대로 가져옴

print('Reviews count:', len(reviews))
print('Length of the first review:', len(reviews[0]))
print('Labels:', set(categories))

Reviews count: 2000
Length of the first review: 4043
Labels: {'pos', 'neg'}


### train set과 test set의 분리

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

적절한 비율로 train set과 test set을 분리하여 저장<br>
train set은 학습에 사용되고, test set은 검증에 사용<br>
default로 shuffle을 함: train set과 test set이 고르게 분포되도록 하기 위함

In [2]:
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
X_train, X_test, y_train, y_test = train_test_split(reviews, categories, test_size=0.2, random_state=10)
# sklearn의 train_test_split 함수는 먼저 data set을 shuffle하고 주어진 비율에 따라 train set과 test set을 나눠 줌
# 위에서는 reviews를 X_train과 X_test로 8:2의 비율로 나누고, categories를 y_train과 y_test로 나눔
# 이 때 X와 y의 순서는 동일하게 유지해서 각 입력값과 label이 정확하게 match되도록 함
# random_state는 shuffle에서의 seed 값으로, 지정한 경우 항상 동일한 결과로 shuffle이 됨

print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

Train set count:  1600
Test set count:  400


### TFIDF 변환

http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
    
CountVectorizer로 Count Vector를 생성하고 TFIDF로 변환하는 대신, text로부터 직접 생성

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
#sklearn에서 제공하는 TfidfVectorizer를 이용
tfidf = TfidfVectorizer().fit(X_train) # X_train을 이용하여 vectorizer를 학습
tfidf #vectorize에서 사용한 매개변수 값들을 확인 -> 현재는 모두 default 값을 사용, 향후 tokenizer, max_features 등을 지정할 수 있음
# 상세한 매개변수 내용은 위 링크를 참조

TfidfVectorizer()

In [4]:
X_train_tfidf = tfidf.transform(X_train) #학습된 vectorizer를 이용하여 train set을 변환
X_train_tfidf.shape # 1600 (review 수) x 36310 (전체 corpus에서 사용된 단어의 수) 크기로 vector set이 생성됨
# matrix 안의 값은 해당 tfidf score임

(1600, 36310)

In [5]:
tfidf = TfidfVectorizer(max_features=2000).fit(X_train) #사용된 단어의 수가 너무 많은 경우, max_feature를 제한하여 학습이 가능
tfidf #max_factures 값이 사용된 것을 볼 수 있음

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=2000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [5]:
X_train_tfidf = tfidf.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_tfidf.shape) # 36310 대신 2000이 된 것을 확인
X_test_tfidf = tfidf.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_tfidf.shape)

Train set dimension: (1600, 36310)
Test set dimension: (400, 36310)


## Naive Bayse Classifier (Scikit)

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [6]:
#나이브 베이즈는 word count를 사용하므로 tfdif가 아닌 count vectorizer를 사용하여 학습 및 변환
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=2000).fit(X_train) #tfidf와 동일하게 max_feature를 제한하여 학습
X_train_cv = cv.transform(X_train) # train set을 변환
print('Train set dimension:', X_train_cv.shape) # 36310 대신 2000이 된 것을 확인
X_test_cv = cv.transform(X_test) # test set을 변환
print('Test set dimension:', X_test_cv.shape)

Train set dimension: (1600, 2000)
Test set dimension: (400, 2000)


In [7]:
from sklearn.naive_bayes import MultinomialNB #sklearn이 제공하는 MultinomialNB 를 사용
NB_clf = MultinomialNB() # 분류기 선언

NB_clf.fit(X_train_cv, y_train) #train set을 이용하여 분류기(classifier)를 학습
#NB_clf.fit(X_train_tfidf, y_train) #tfidf 값을 사용할 수도 있으나, NB 이론에 맞지 않음

MultinomialNB()

In [8]:
print('Train set score: {:.3f}'.format(NB_clf.score(X_train_cv, y_train))) #train set에 대한 예측정확도를 확인
print('Test set score: {:.3f}'.format(NB_clf.score(X_test_cv, y_test))) #test set에 대한 예측정확도를 확인
#실제로 필요한 것은 test set에 대한 예측정확도이나, 과적합 (overfitting)의 문제가 있는지를 보기 위해 train set에 대한 예측정확도를 같이 확인
#print('Train set score: {:.3f}'.format(NB_clf.score(X_train_tfidf, y_train)))
#print('Test set score: {:.3f}'.format(NB_clf.score(X_test_tfidf, y_test)))

Train set score: 0.864
Test set score: 0.775


In [9]:
#여러 문장에 대해 count vectorier로 변환 후 학습된 분류기로 결과를 예측
print(NB_clf.predict(cv.transform(['the story was unimaginative', 'the plot was ludicrous', 'kate winslet is accessible'])))

#위 첫째 문장에서 story를 actor로 바꿔서 예측
print(NB_clf.predict(cv.transform(['the actor was unimaginative'])))
#동일한 형용사라도 대상에 따라 결과가 바뀔 수 있음

['pos' 'neg' 'pos']
['neg']


## Logistic Regression (Scikit)

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

예측하고자 하는 값 혹은 label이 연속적인 값이 아니고 분류(class)일 때 사용하는 regression 방법<br>
분류는 binary인 경우와 multi-class인 경우가 있음<br>
지금은 positive와 negative 두 class 중에서 선택하므로 binary classification 문제임

In [10]:
from sklearn.linear_model import LogisticRegression #sklearn이 제공하는 logistic regression을 사용

#count vector에 대해 regression을 해서 NB와 비교
LR_clf_cv = LogisticRegression() #분류기 선언
LR_clf_cv.fit(X_train_cv, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf_cv.score(X_train_cv, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf_cv.score(X_test_cv, y_test))) # test data에 대한 예측정확도
# count vector를 이용한 regression 결과가 tfidf보다 더 좋게 나옴
# 보통은 tfidf가 더 좋은 결과를 보이는데, 이와 같이 상황에 따라 다른 결과가 나오기도 함
# 지금은 train data의 수가 1,600개인데 비해, 추정해야 하는 parameter의 수가 2,000개로 sample 수가 학습에 부족한 상황, 
# 따라서 예상 못한 다양한 결과가 나올 수 있음
# 좀더 data가 많은 상황에서의 test가 필요

Train set score: 1.000
Test set score: 0.825


C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
#tfidf vector를 이용해서 분류기 학습
LR_clf = LogisticRegression() #분류기 선언
LR_clf.fit(X_train_tfidf, y_train) # train data를 이용하여 분류기를 학습
print('Train set score: {:.3f}'.format(LR_clf.score(X_train_tfidf, y_train))) # train data에 대한 예측정확도 
print('Test set score: {:.3f}'.format(LR_clf.score(X_test_tfidf, y_test))) # test data에 대한 예측정확도
# NB에 비해 더 좋은 결과가 나오는 것을 확인

Train set score: 0.960
Test set score: 0.828


In [12]:
# NB 분류기에서 사용했던 예제로 결과 확인, 실제로 결과가 더 나아졌음을 확인할 수 있음
LR_clf.predict(tfidf.transform(['the story was unimaginative', 'the plot was ludicrous', 'kate winslet is accessible']))

array(['neg', 'neg', 'neg'], dtype='<U3')

In [14]:
from sklearn.linear_model import RidgeClassifier
ridge_clf = RidgeClassifier() #릿지 분류기 선언
ridge_clf.fit(X_train_tfidf, y_train) #학습
print('Train set score: {:.3f}'.format(ridge_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(ridge_clf.score(X_test_tfidf, y_test)))
# 일반적으로 ridge regression을 쓰면 쓰지 않은 경우보다 train data에 대한 예측정확도는 떨어지고 test data는 올라가게 됨
# 여기서는 train set에 대한 예측정확도가 같이 상승하는 진귀한 경우가 발생

Train set score: 0.998
Test set score: 0.835


In [15]:
from sklearn.linear_model import LogisticRegression
import numpy as np
lasso_clf = LogisticRegression(penalty='l1', solver='liblinear') # Lasso는 동일한 LogisticRegression을 사용하면서 매개변수로 지정
lasso_clf.fit(X_train_tfidf, y_train) # train data로 학습
print('Train set score: {:.3f}'.format(lasso_clf.score(X_train_tfidf, y_train)))
print('Test set score: {:.3f}'.format(lasso_clf.score(X_test_tfidf, y_test)))
print('Used features count: {}'.format(np.sum(lasso_clf.coef_ != 0)), 'out of', X_train_tfidf.shape[1]) 
# parameter 혹은 coefficient 중에서 0이 아닌 것들의 개수를 출력
# 2000개 중에서 78개만 선택된 것을 볼 수 있음
# 예측률은 rigde나 일반 logistic에 비해 떨어지지만, 실제로 영향을 미치는 단어들이 어떤 것들인지 확인할 수 있다는 장점이 있음

Train set score: 0.770
Test set score: 0.755
Used features count: 41 out of 36310


In [16]:
print(len(tfidf.vocabulary_)) # tfidf에 사용된 단어의 수
tfidf_voca = tfidf.get_feature_names() # tfidf에서 단어이름을 가져옴
tfidf_voca[:10] # 앞 10개를 출력

36310


['00', '000', '0009f', '007', '00s', '03', '04', '05', '05425', '10']

In [17]:
print((lasso_clf.coef_ != 0)[0].tolist()[:100]) 
# lasso에 사용된 단어들 중 coefficient의 사용여부를 리스트로 변환하여 앞부터 100개를 출력해 봄

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [18]:
selected_features = []
for i, sign in enumerate((lasso_clf.coef_ != 0)[0].tolist()):
    if sign: selected_features.append(tfidf_voca[i]) #사용여부가 True인 단어들만 selected_features에 저장

In [19]:
print(selected_features) #78개의 선택된 단어들을 출력 - 즉 positive, negative에 결정적 영향을 미치는 단어들
len(selected_features)

['also', 'and', 'any', 'as', 'bad', 'batman', 'boring', 'could', 'great', 'harry', 'have', 'his', 'is', 'jackie', 'julie', 'life', 'most', 'movie', 'mulan', 'no', 'nothing', 'on', 'only', 'plot', 'poor', 'script', 'see', 'shrek', 'stupid', 'supposed', 'the', 'there', 'this', 'to', 'truman', 'unfortunately', 'very', 'war', 'well', 'why', 'worst']


41

In [20]:
tfidf_voca[(lasso_clf.coef_ != 0)[0].tolist().index(True)]

'also'

In [21]:
print([tfidf_voca[i] for i, j in enumerate((lasso_clf.coef_ != 0)[0].tolist()) if j]) #선택된 단어들을 출력하는 또다른 방법

['also', 'and', 'any', 'as', 'bad', 'batman', 'boring', 'could', 'great', 'harry', 'have', 'his', 'is', 'jackie', 'julie', 'life', 'most', 'movie', 'mulan', 'no', 'nothing', 'on', 'only', 'plot', 'poor', 'script', 'see', 'shrek', 'stupid', 'supposed', 'the', 'there', 'this', 'to', 'truman', 'unfortunately', 'very', 'war', 'well', 'why', 'worst']


In [22]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42) #압축할 component의 수 지정
svd.fit(X_train_tfidf)  # train data로 학습 -> unsupervised이므로 y가 필요 없음
print(svd.explained_variance_ratio_)  #계산된 각 component가 설명하는 분산의 비율
print(svd.explained_variance_ratio_.sum())  #선택된 component들이 설명하는 분산의 합 -> 선택한 component의 수에 따라 달라짐
# 현재 결과에서는 100개의 선택된 component가 전체 분산의 34% 정도를 설명하고 있음
print(svd.singular_values_) #위 그림에서 오른쪽 가운데 대각행렬의 값
print(svd.components_.shape) # 위 그림에서 가장 오른쪽 행렬의 값

[0.0108655  0.00688034 0.00557916 0.00430563 0.00343487 0.00327273
 0.00318433 0.00303562 0.00285556 0.0027845  0.0027095  0.00261101
 0.00254919 0.00253836 0.00250339 0.00244937 0.0024094  0.00235419
 0.00231941 0.00227256 0.00219487 0.0021724  0.00215844 0.00209706
 0.00207466 0.00207003 0.00200618 0.0019982  0.00198273 0.00195395
 0.00193161 0.00190787 0.00187946 0.00186988 0.00185076 0.001845
 0.00181044 0.00180344 0.00178382 0.00177683 0.00176379 0.00175012
 0.00174169 0.0017237  0.00170265 0.00169587 0.00168154 0.00166108
 0.00165473 0.00163217 0.0016171  0.00161332 0.00160262 0.00159353
 0.00158349 0.00158262 0.0015657  0.00155384 0.0015489  0.00154028
 0.00152711 0.00152339 0.00150462 0.00149873 0.00149436 0.00148546
 0.00147511 0.00147123 0.00145775 0.0014546  0.0014493  0.00143716
 0.00142815 0.00142004 0.0014128  0.00140321 0.00139648 0.00139311
 0.00138279 0.00138069 0.00137474 0.00136622 0.00135729 0.00134996
 0.00134649 0.00133292 0.001332   0.00132222 0.0013101  0.001305

In [23]:
X_train_svd = svd.transform(X_train_tfidf) #선택된 component를 이용하여 2,000개의 feature로부터 feature extract (dimension reduce)
print(X_train_tfidf.shape) #축소하기 전의 차원
print(X_train_svd.shape) #축소된 후의 차원, 2000 -> 100개로 줄어 있음

(1600, 36310)
(1600, 100)


#### LSA를 이용한 Logistic Regression

In [24]:
X_test_svd = svd.transform(X_test_tfidf)

from sklearn.linear_model import LogisticRegression
SVD_clf = LogisticRegression()
SVD_clf.fit(X_train_svd, y_train) #축소된 값으로 학습
print('Train set score: {:.3f}'.format(SVD_clf.score(X_train_svd, y_train)))
print('Test set score: {:.3f}'.format(SVD_clf.score(X_test_svd, y_test)))
# NB, Lasso보다 좋지만 Ridge, 일반보다는 나쁜 값
# 상황에 따라 달라질 수 있음

Train set score: 0.838
Test set score: 0.790
